In [1]:
# implementation of LRU (least recently used cache) using linked lists approach.

In [2]:
from __future__ import annotations

In [3]:
class Node:
    def __init__(self, key, value):
        self.key = key 
        self.value = value
        self.prev = None
        self.next = None

    def __repr__(self):
        return f"{self.key}:{self.value}"
        

In [56]:
class LRUCache:
    def __init__(self, capacity):
        self.capacity = capacity
        assert self.capacity > 0; 'Capacity must be 1 atleast, cannot be zero'
        self.head = Node(0, 0)
        self.tail = Node(0, 0)
        self.head.next = self.tail
        self.tail.prev = self.head
        self._cache = {}

    def __repr__(self):
        nodes = []
        
        curr = self.head.next
        while curr != self.tail:
            nodes.append(f'[{curr.key}:{curr.value}]')
            curr = curr.next
        return '<->'.join(nodes)

    # -------- Helpers --------
    def _insert_first(self, node: Node):
        '''Adds a node at the start of the chain'''
        node.prev = self.head
        node.next = self.head.next
        self.head.next.prev = node
        self.head.next = node
        self._cache[node.key] = node
        
    def _remove(self, node: Node):
        '''Removes a specific node from the chain'''
        if node is self.head or node is self.tail:
            return 
        prev_node = node.prev
        next_node = node.next
        if prev_node == None or next_node == None:
            raise ValueError('None node at prev or next')
        prev_node.next = next_node
        next_node.prev = prev_node
        node.prev = node.next = None

    def _remove_last(self):
        '''Removes the last node from the chain'''
        if self.head.next == self.tail:
            return None
        last = self.tail.prev 
        self._remove(last)
        return last

    # -------- Main Functions --------
    def get(self, key):
        '''Main get action, where the node with the key value "key" is fetched and upgraded to MRU position'''
        node = self._cache[key]
        if node is None:
            return None
        self._remove(node)          # unlink Node
        self._insert_first(node)    # add to MRU
        return node.value

    def put(self, key, value):
        '''Put/Update value at key position in cache, remove LRU (self.tail) on overflow'''

        if key in self._cache:
            node = self._cache[key]
            node.value = value       # update
            self._remove(node)       # unlink Node
            self._insert_first(node) # add to MRU

        node = Node(key, value)
        self._insert_first(node)
        self._cache[key] = node

        # handling overflow
        if len(self._cache) > self.capacity:
            lru = self._remove_last()
            del self._cache[lru.key]
        

## Testing API

In [49]:
x = LRUCache(capacity=2)
x._insert_first(Node(1, 'A'))
x._insert_first(Node(2, 'B'))
x._insert_first(Node(3, 'C'))
x

[3:C]<->[2:B]<->[1:A]

In [50]:
n = x._cache[1]
x._remove(n)
x

[3:C]<->[2:B]

In [51]:
x._cache

{1: 1:A, 2: 2:B, 3: 3:C}

In [52]:
key = 2
x._cache[2] 

2:B

In [53]:
x.get(key)
x

[2:B]<->[3:C]

In [54]:
x.put(4, 'R')

In [55]:
x

[4:R]<->[2:B]

## SANITY CHECKING

In [63]:
c = LRUCache(2)
c.put(1, "a"); c.put(2, "b")
assert c.get(1) == "a"        # 1 is MRU
c.put(3, "c")                 # evicts 2
print(c, 'evicts 2')
assert c.get(1) == "a"
assert c.get(3) == "c"
c.put(4, "d")                 # evicts 1
print(c, 'evicted 1')
assert c.get(3) == "c"
assert c.get(4) == "d"
print("OK:", c)

[3:c]<->[1:a] evicts 2
[4:d]<->[3:c] evicted 1
OK: [4:d]<->[3:c]
